# Лабораторная работа №1

### 1. Численные методы решения СЛАУ

In [1]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Методы численного решения СЛАУ
from comp_math.linear_algebra.sla_solvers.SLA_solvers_registry import SLASolverRegistry
from comp_math.linear_algebra.objects.matrix import Matrix
from comp_math.linear_algebra.objects.vector import Vector

Добавим СЛАУ с которой будет работать

In [2]:
# Из задачника Аристова-Лобанова пункт Д)
A = np.zeros((100, 100))
b = np.zeros(100)
n = len(b)

for k in range(n):
    start_j = max(0, k - 4)
    end_j = min(99, k + 4)
    
    for j in range(start_j, end_j + 1):
        if j == k:
            A[k, j] = 10
        else:
            A[k, j] = 1
    
    b[k] = k + 1

Выведем матрицу, чтобы убедиться в её корректности

In [3]:
for i in range(0, len(A)):
    print(A[i])

[10.  1.  1.  1.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
[ 1. 10.  1.  1.  1.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
[ 1.  1. 10.  1.  1.  1.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.

Перечислим методы для решения СЛАУ, которые будут использоваться

In [4]:
methods = [
        (SLASolverRegistry.create_solver("gauss"), "Метод Гаусса"),
        (SLASolverRegistry.create_solver("jacobi"), "Метод Якоби"),
        (SLASolverRegistry.create_solver("zeidel"), "Метод Зейделя"),
        (SLASolverRegistry.create_solver("relaxation"), "Метод релаксации"),
        (SLASolverRegistry.create_solver("cg"), "Сопряжённые градиенты"),
        (SLASolverRegistry.create_solver("bcg"), "Бисопряжённые градиенты"),
        (SLASolverRegistry.create_solver("sbcg"), "Стабилизированный BCG")
    ]

Решим этими методами нашу СЛАУ

In [5]:
results = []
for method, name in methods:
    results.append(method.solve(Matrix(A), Vector(b)))

[0.0229214  0.09689605 0.16487571 0.22670602 0.28230826 0.33422809
 0.38818309 0.44352717 0.49957984 0.55564296 0.61130107 0.66677899
 0.722232   0.77774012 0.83330511 0.88888183 0.94445003 1.00000675
 1.05555813 1.11111052 1.16666527 1.22222144 1.27777775 1.3333336
 1.38888909 1.44444449 1.49999996 1.55555551 1.61111109 1.66666667
 1.72222223 1.77777778 1.83333333 1.88888889 1.94444444 2.
 2.05555556 2.11111111 2.16666667 2.22222222 2.27777778 2.33333333
 2.38888889 2.44444444 2.5        2.55555556 2.61111111 2.66666667
 2.72222222 2.77777778 2.83333333 2.88888889 2.94444444 3.
 3.05555555 3.11111111 3.16666667 3.22222222 3.27777778 3.33333334
 3.38888888 3.44444442 3.49999998 3.55555557 3.61111119 3.6666668
 3.72222225 3.77777748 3.8333327  3.88888846 3.94444536 4.00000287
 4.05555861 4.11110933 4.16665466 4.22220464 4.27777582 4.33337627
 4.38898044 4.44449555 4.4998623  4.55515455 4.61067178 4.66701868
 4.72390386 4.78014559 4.83372121 4.88185838 4.93252886 4.99368677
 5.072974   5